In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
# use catboost train on gpu

In [26]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

#### Some methods below are implemented because the data is very large for my current setup and I run out of memory

In [27]:
pd.options.display.max_columns = 100

In [28]:
# All the variables here used in create_df() below
h = 28 # forecast horizon (days)
max_lags = 57 # number of observations to shift by
tr_last = 1913 # last training day
fday = datetime(2016,4, 25) # first day to forecast
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [29]:
def create_df(is_train = True, nrows = None, first_day = 1200):
    """
    create_df() creates a training or testing df from a wide table.
    The majority of the code is converting the columns into efficient datatypes
    and getting it into the correct format (melting and merging).
    """
    prices = pd.read_csv(r'C:\Users\james\Desktop\Data Science Projects\Predict Future Sales\m5-forecasting-accuracy\sell_prices.csv', dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv(r'C:\Users\james\Desktop\Data Science Projects\Predict Future Sales\m5-forecasting-accuracy\calendar.csv', dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    df = pd.read_csv(r'C:\Users\james\Desktop\Data Science Projects\Predict Future Sales\m5-forecasting-accuracy\sales_train_validation.csv', 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            df[col] = df[col].cat.codes.astype("int16")
            df[col] -= df[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            df[f"d_{day}"] = np.nan
    
    df = pd.melt(df,
                  id_vars = catcols,
                  value_vars = [col for col in df.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    df = df.merge(cal, on= "d", copy = False)
    df = df.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return df

In [30]:
# sets which day to start taking our training data from
FIRST_DAY = 350

In [31]:
df = create_df(is_train=True, first_day= FIRST_DAY)
df.shape

(40718219, 22)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory us

In [34]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,3.97
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,4.34
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,2.48
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,0.50
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,1.77


In [35]:
# df['wday'] and df['weekday'] have the same information so we will drop one of them
df[['wday','weekday']]

,wday,weekday
0,7,0
1,7,0
2,7,0
3,7,0
4,7,0
...,...,...
40718214,2,3
40718215,1,2
40718216,2,3
40718217,1,2


In [36]:
df[['id','sales']].head(3000).groupby('id')['sales'].shift(7)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2995   NaN
2996   NaN
2997   NaN
2998   NaN
2999   NaN
Name: sales, Length: 3000, dtype: float32

In [46]:
def create_features(df):
    """
    create_features() adds lags, rolling features and time variables to the dataframe.
    """
    lags = [7, 28] # one week and one month roughly, can add more lags here one year would be good but we have memory issues
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            df[f"rmean_{lag}_{win}"] = df[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in df.columns:
            df[date_feat_name] = df[date_feat_name].astype("int16")
        else:
            df[date_feat_name] = getattr(df["date"].dt, date_feat_func).astype("int16")

In [47]:
%%time

create_features(df)
df.shape

Wall time: 7min 44s


(39041269, 31)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 28 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
 22  lag_7         float32       
 23  lag_28        float32       
 

In [40]:
# drops our first days lagged (doesn't affect prediction)
df.dropna(inplace = True)
df.shape

(39041269, 28)

In [41]:
# gets rid of uneeded columns
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

### from here I would do splitting of the data set for cross validation testing, however due to the size of the dataset I will be stopping here. 
### training on a decent subset of this data would take a very long time let alone the whole dataset.
### Algorithm selection would involve trying several regression algorithms to see which is the most predictive using the cross validation processes described below.
### The splitting would be done to preserve the order of the time series (something like nested cross val or walk-forward). Afterwords would be hyperparameter tuning (likely using Bayesian optimisation) then retrain on whole dataset and predict.
### I will be exploring options how to handle larger data in a speedier fashion in other notebooks or via cloud services.